## Manufacturer Analysis using machine learning
The goal of this project is to find the most influential features from manufacturer profile using machine learning models linear regression.

In [1]:
#Use match_helper.py file to join the SQL table
import pandas as pd
import math
import match_helper as helper
from imp import reload
import numpy as np
reload(helper)
dbaccess = helper.get_dbaccess()
buyer_access = dbaccess + ['buyer']
manufacturer_access = dbaccess + ['manufacturer']
shared_access = dbaccess + ['shared']
crm_access = dbaccess + ['crm']
logging_access = dbaccess + ['logging']
messaging_access = dbaccess + ['messaging']
auth_access = dbaccess + ['auth']

/Users/glen/Desktop/Foursource/work/Manufacturer analysis/


### Extract manufacturer features
The main features are extracted from Country - Fill Out Detail Tableau. (database: manufacturer)  


In [2]:
### Paste the photo of data structure here

In [3]:
# Create manufacturer table m_df (database: manufacturer)
reload(helper)
manufacturer = """
With manufacturer AS (SELECT
          m.id_manufacturer
        , m.name AS manufacturer_name
        , CASE WHEN m.profile_image != '' THEN 'yes' ELSE 'no' END AS has_profile_image
        , m.city AS manufacturer_city
        , MIN(country.name) AS manufacturer_country
        , CASE WHEN m.fk_manufacturer_status = 1 THEN 'Active' ELSE 'Deactivated' END AS manufacturer_status
        , m.created_at AS manufacturer_created_at
        , to_char(m.created_at::timestamp WITH TIME ZONE, 'iyyy"W"IW') AS m_created_year_week
        , to_char(m.created_at::timestamp WITH TIME ZONE, 'yyyy"M"mm') AS m_created_year_month
        , to_char(m.created_at::timestamp WITH TIME ZONE, 'yyyy') AS m_created_year
        , CASE 
                WHEN COUNT(DISTINCT(mt.name)) > 1 
                        THEN string_agg(DISTINCT(mt.name), ', ') 
                ELSE string_agg(DISTINCT(mt.name), '') 
          END AS manufacturer_types
        , CASE WHEN string_agg(DISTINCT(mt.name), '') IS NOT NULL THEN 'yes' ELSE 'no' END AS manufacturer_type_filled
        , CASE 
                WHEN COUNT(DISTINCT(c.id_contact)) > 1
                        THEN string_agg(DISTINCT(c.first_name || ' ' || c.last_name), ', ')
                ELSE string_agg(DISTINCT(c.first_name || ' ' || c.last_name), '')
          END AS all_contacts
        , COUNT(DISTINCT(c.id_contact)) AS contact_count
        , CASE
                WHEN COUNT(DISTINCT(c.id_contact)) > 1
                        THEN string_agg(DISTINCT(c.email), ', ')
                ELSE string_agg(DISTINCT(c.email), '')
          END AS emails
        , pf.filled_data
        , (pf.filled_data ->> 'all')::jsonb ->> 'percentage' AS all_fields_percentage
        , (pf.filled_data ->> 'mandatory')::jsonb ->> 'percentage' AS mandatory_fields_percentage
        , pf.filled_mandatory_reached
        , pf.filled_mandatory_reached_at
        , MIN(po.min_qty) AS po_min_qty
        , MIN(po.avg_qty) AS po_avg_qty
        , MIN(po.max_qty) AS po_max_qty
        , COUNT(DISTINCT(s.fk_service_type)) AS count_parent_services
        , COUNT(DISTINCT(ms.fk_service)) AS count_serives
        , CASE
                WHEN COUNT(DISTINCT(ms.fk_service)) > 1
                        THEN string_agg(DISTINCT(s.name), ', ')
                ELSE string_agg(DISTINCT(s.name), '')
          END AS services
        , (
                SELECT 
                        lt.minimum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 1
          ) AS on_stock_min
        , (
                SELECT 
                        lt.maximum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 1
          ) AS on_stock_max
        , CASE 
                WHEN (
                        SELECT 
                                lt.minimum + lt.maximum
                        FROM lead_time AS lt 
                        WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 1
                ) IS NOT NULL
                THEN 'yes'
                ELSE 'no'
          END AS on_stock_filled
        , (
                SELECT 
                        lt.minimum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 2
          ) AS made_to_order_min
        , (
                SELECT 
                        lt.maximum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 2
          ) AS made_to_order_max  
        , CASE 
                WHEN (
                        SELECT 
                                lt.minimum + lt.maximum
                        FROM lead_time AS lt 
                        WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 2
                ) IS NOT NULL
                THEN 'yes'
                ELSE 'no'
          END AS made_to_order_filled
        , (
                SELECT 
                        lt.minimum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 3
          ) AS sample_prototype_min
        , (
                SELECT 
                        lt.maximum 
                FROM lead_time AS lt 
                WHERE lt.fk_manufacturer = m.id_manufacturer AND fk_lead_time_type = 3
          ) AS sample_prototype_max 
        , string_agg(DISTINCT( '(' || lt.minimum || '-' || lt.maximum || '-' || ltt.title || ')' ), ' / ') AS lead_time_detail
        , CASE WHEN COUNT(DISTINCT l.id_location) > 0 THEN 'yes' ELSE 'no' END AS has_location
        , COUNT(DISTINCT l.id_location) AS location_count
FROM manufacturer AS m
LEFT JOIN profile_fill AS pf ON pf.fk_manufacturer = m.id_manufacturer
LEFT JOIN country ON country.id_country = m.fk_country
LEFT JOIN manufacturer_manufacturer_type AS mtr ON mtr.fk_manufacturer = m.id_manufacturer
LEFT JOIN manufacturer_type AS mt ON mt.id_manufacturer_type = mtr.fk_manufacturer_type
LEFT JOIN manufacturer_user AS mu ON mu.fk_manufacturer = m.id_manufacturer
LEFT JOIN contact AS c ON c.id_contact = mu.fk_contact
LEFT JOIN purchase_order_size AS po ON po.fk_manufacturer = m.id_manufacturer
LEFT JOIN lead_time AS lt ON lt.fk_manufacturer = m.id_manufacturer
LEFT JOIN lead_time_type AS ltt ON ltt.id_lead_time_type = lt.fk_lead_time_type
LEFT JOIN manufacturer_service AS ms ON ms.fk_manufacturer = m.id_manufacturer
LEFT JOIN service AS s ON s.id_service = ms.fk_service 
LEFT JOIN location AS l ON l.fk_manufacturer = m.id_manufacturer AND l.active = TRUE
GROUP BY m.id_manufacturer, pf.id_profile_fill
ORDER BY m.id_manufacturer )


SELECT id_manufacturer,
        manufacturer_name,
        has_profile_image,
        manufacturer_city,
        manufacturer_country,
        manufacturer_status,
        manufacturer_type_filled,
        m_created_year_week,
        m_created_year_month,
        m_created_year,
        contact_count,
        all_fields_percentage,
        mandatory_fields_percentage,
        filled_mandatory_reached,
        po_min_qty,
        po_avg_qty,
        po_max_qty,
        count_parent_services,
        count_serives,
        on_stock_min,
        on_stock_max,
        made_to_order_min,
        made_to_order_max,
        sample_prototype_min,
        sample_prototype_max,
        location_count AS factory_location_count
        
FROM manufacturer

"""
m_df = helper.query_retrieval(manufacturer, manufacturer_access)
m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,po_max_qty,count_parent_services,count_serives,on_stock_min,on_stock_max,made_to_order_min,made_to_order_max,sample_prototype_min,sample_prototype_max,factory_location_count
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,1000000.0,2,7,NaN,NaN,NaN,NaN,NaN,NaN,2
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,50000.0,4,28,45.0,60.0,60.0,90.0,15.0,30.0,1
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,400000.0,2,7,50.0,70.0,60.0,90.0,20.0,30.0,9
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,500000.0,4,27,47.0,59.0,80.0,90.0,5.0,9.0,1


In [4]:
# create m_product_range_count table (database: shared)
m_product = """
SELECT
          t.fk_manufacturer AS id_manufacturer
        , COUNT(DISTINCT t.fk_ct1 || '-' || t.fk_ct2 || '-' || t.fk_ct3) AS count_parent_product_ranges
        , SUM(t.sub_product_category_count) AS sub_product_category_count
FROM (
        SELECT
                  mo.fk_manufacturer
                , ct.fk_ct1
                , ct.fk_ct2
                , ct.fk_ct3
                , COUNT(mo.id_manufacturer_offering) AS sub_product_category_count
        FROM manufacturer_offering AS mo
        LEFT JOIN category_tree AS ct ON ct.id_category_tree = mo.fk_category_tree
        WHERE mo.active = TRUE
        GROUP BY mo.fk_manufacturer, ct.fk_ct1, ct.fk_ct2, ct.fk_ct3
        ORDER BY mo.fk_manufacturer
) AS t
GROUP BY t.fk_manufacturer
"""
m_product_df = helper.query_retrieval(m_product, shared_access)
m_product_df[:5]

,id_manufacturer,count_parent_product_ranges,sub_product_category_count
0,1,2,7.0
1,2,3,17.0
2,4,4,23.0
3,5,5,32.0
4,6,2,4.0


In [5]:
# Create m_brand_product_range_count table (database: shared)

m_brand = """
SELECT
         t.fk_manufacturer AS id_manufacturer
       , SUM(t.brand_count) AS brand_count
       , COUNT(DISTINCT fk_ct1 || '-' || fk_ct2 || '-' || fk_ct3) AS count_parent_brands
FROM (
        SELECT
                  fk_manufacturer
                , fk_ct1
                , fk_ct2
                , fk_ct3
                , COUNT(fk_brand) AS brand_count
        FROM manufacturer_customer AS mc
        WHERE mc.fk_brand IS NOT NULL AND active = TRUE
        GROUP BY fk_manufacturer, fk_ct1, fk_ct2, fk_ct3
        ORDER BY fk_manufacturer
) AS t
GROUP BY t.fk_manufacturer

"""
m_brand_df = helper.query_retrieval(m_brand, shared_access)
m_brand_df[:5]

,id_manufacturer,brand_count,count_parent_brands
0,1,1.0,1
1,2,19.0,3
2,4,9.0,4
3,5,40.0,5
4,6,3.0,2


#### External join m_product_df and m_brand_df on m_df table

In [6]:
new_m_df = pd.merge(m_df, m_brand_df, on='id_manufacturer', how = 'left')
new_m_df = pd.merge(new_m_df, m_product_df, on='id_manufacturer', how = 'left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,on_stock_max,made_to_order_min,made_to_order_max,sample_prototype_min,sample_prototype_max,factory_location_count,brand_count,count_parent_brands,count_parent_product_ranges,sub_product_category_count
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,2,1.0,1.0,2.0,7.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,60.0,60.0,90.0,15.0,30.0,1,19.0,3.0,3.0,17.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,70.0,60.0,90.0,20.0,30.0,9,9.0,4.0,4.0,23.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,59.0,80.0,90.0,5.0,9.0,1,40.0,5.0,5.0,32.0


#### External join material & material_type count  (share database)

In [7]:
# Create material and material type table and join together with new_m_df
m_material = """
---material & material_type count

SELECT fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT m.id_material) AS material_count,
        COUNT(DISTINCT m.material_type) AS material_type_count
        
FROM(
        SELECT id_material,
                m.name AS material_name,
                fk_material_type AS material_type,
                mt.name AS material_type_name
        FROM material AS m
        LEFT JOIN material_type AS mt ON m.fk_material_type = mt.id_material_type
        WHERE active = true AND verified = true
) AS m
LEFT JOIN manufacturer_material AS mm ON m.id_material = mm.fk_material
WHERE mm.active = true
GROUP BY fk_manufacturer;
"""
m_material_df = helper.query_retrieval(m_material, shared_access)
new_m_df = pd.merge(new_m_df, m_material_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,made_to_order_max,sample_prototype_min,sample_prototype_max,factory_location_count,brand_count,count_parent_brands,count_parent_product_ranges,sub_product_category_count,material_count,material_type_count
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,NaN,NaN,NaN,2,1.0,1.0,2.0,7.0,2.0,1.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,90.0,15.0,30.0,1,19.0,3.0,3.0,17.0,3.0,1.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,90.0,20.0,30.0,9,9.0,4.0,4.0,23.0,12.0,1.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,90.0,5.0,9.0,1,40.0,5.0,5.0,32.0,22.0,2.0


In [8]:
# Create fabric and fabric type table and join together with new_m_df
m_fabric = """
---fabric & fabric_type count

SELECT fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT f.id_fabric_type) AS fabric_count,
        COUNT(DISTINCT f.fabric_type) AS fabric_type_count
        
FROM(
        SELECT id_fabric_type,
                f.name AS fabric_name,
                fk_fabric_type_group AS fabric_type,
                ft.name AS fabric_type_name
        FROM fabric_type AS f
        LEFT JOIN fabric_type_group AS ft ON f.fk_fabric_type_group = ft.id_fabric_type_group
        WHERE active = true AND verified = true
) AS f
LEFT JOIN manufacturer_fabric_type AS mft ON f.id_fabric_type = mft.fk_fabric_type
WHERE mft.active = true
GROUP BY fk_manufacturer;
"""
m_fabric_df = helper.query_retrieval(m_fabric, shared_access)
new_m_df = pd.merge(new_m_df, m_material_df, on = 'id_manufacturer', how='left')
new_m_df[:5]


,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,sample_prototype_max,factory_location_count,brand_count,count_parent_brands,count_parent_product_ranges,sub_product_category_count,material_count_x,material_type_count_x,material_count_y,material_type_count_y
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,NaN,2,1.0,1.0,2.0,7.0,2.0,1.0,2.0,1.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,30.0,1,19.0,3.0,3.0,17.0,3.0,1.0,3.0,1.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,30.0,9,9.0,4.0,4.0,23.0,12.0,1.0,12.0,1.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,9.0,1,40.0,5.0,5.0,32.0,22.0,2.0,22.0,2.0


In [9]:
# Create treatment and treatment type table and join together with new_m_df
m_treatment = """
---treatment & treatment_type count

SELECT fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT t.id_treatment) AS treatment_count,
        COUNT(DISTINCT t.treatment_type) AS treatment_type_count
        
FROM(
        SELECT id_treatment,
                t.name AS treatment_name,
                fk_treatment_type AS treatment_type,
                tt.name AS treatment_type_name
        FROM treatment AS t
        LEFT JOIN treatment_type AS tt ON t.fk_treatment_type = tt.id_treatment_type
        WHERE active = true AND verified = true
) AS t
LEFT JOIN manufacturer_treatment AS mt ON t.id_treatment = mt.fk_treatment
WHERE mt.active = true
GROUP BY fk_manufacturer;
"""
m_treatment_df = helper.query_retrieval(m_treatment, shared_access)
new_m_df = pd.merge(new_m_df, m_treatment_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,brand_count,count_parent_brands,count_parent_product_ranges,sub_product_category_count,material_count_x,material_type_count_x,material_count_y,material_type_count_y,treatment_count,treatment_type_count
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,1.0,1.0,2.0,7.0,2.0,1.0,2.0,1.0,5.0,1.0
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,19.0,3.0,3.0,17.0,3.0,1.0,3.0,1.0,27.0,1.0
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,9.0,4.0,4.0,23.0,12.0,1.0,12.0,1.0,45.0,1.0
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,40.0,5.0,5.0,32.0,22.0,2.0,22.0,2.0,37.0,1.0


#### External join Employees number, Revenue/Year, Pieces/Year, reference customer brand count  (manufacturer,shared database)

In [10]:
# About: Employees number, Revenue / Year, Pieces / Year from manufacturer/manufacturer_key_factor

m_key = """
SELECT s.id_manufacturer,
        MIN( CASE WHEN s.id_key_factor = 1 AND s.value != '' THEN s.value ELSE null END ) AS num_Employee,
        MIN( CASE WHEN s.id_key_factor = 2 AND s.value != '' THEN s.value ELSE null END ) AS revenue_per_year,
        MIN( CASE WHEN s.id_key_factor = 3 AND s.value != '' THEN s.value ELSE null END ) AS pieces_per_year
FROM(
        SELECT m.fk_manufacturer AS id_manufacturer,
                m.value,
                k.name,
                k.id_key_factor
        FROM manufacturer_key_factor AS m
        LEFT JOIN key_factor AS k ON m.fk_key_factor = k.id_key_factor
        ) AS s
GROUP BY s.id_manufacturer
ORDER BY s.id_manufacturer
"""
m_key_df = helper.query_retrieval(m_key, manufacturer_access)
new_m_df = pd.merge(new_m_df, m_key_df, on = 'id_manufacturer', how='left')
new_m_df[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,sub_product_category_count,material_count_x,material_type_count_x,material_count_y,material_type_count_y,treatment_count,treatment_type_count,num_employee,revenue_per_year,pieces_per_year
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,7.0,2.0,1.0,2.0,1.0,5.0,1.0,60000,3000000089,4000000000
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,17.0,3.0,1.0,3.0,1.0,27.0,1.0,900,None,3600000
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,23.0,12.0,1.0,12.0,1.0,45.0,1.0,20000,300000000,95000000
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,32.0,22.0,2.0,22.0,2.0,37.0,1.0,150,80000000,18000000


In [11]:
# About: reference customer from shared/manufacturer_brand count

m_ref = """
SELECT m.fk_manufacturer AS id_manufacturer,
        COUNT(m.fk_brand) AS ref_brand_count
FROM manufacturer_brand AS m
LEFT JOIN brand AS b ON m.fk_brand = b.id_brand
WHERE m.active = true AND b.verified = true
GROUP BY m.fk_manufacturer
ORDER BY m.fk_manufacturer;
"""
m_ref_df = helper.query_retrieval(m_ref, shared_access)
new_m_df = pd.merge(new_m_df, m_ref_df, on = 'id_manufacturer', how='left')
new_m_df.shape

(1300, 40)

In [12]:
# About: description

m_des = """
SELECT m.id_manufacturer AS id_manufacturer,
CASE WHEN m.description = '' THEN 'no' ELSE 'yes' END AS has_description
FROM manufacturer AS m
where m.description IS NOT NULL;
"""
m_des_df = helper.query_retrieval(m_des, manufacturer_access)
new_m_df = pd.merge(new_m_df, m_des_df, on = 'id_manufacturer', how='left')
new_m_df[:5]
new_m_df.shape

(1300, 41)

#### External join garment production and quality assurance table (manufacturer database)

In [13]:
# Machinery: fabric production, garment production and quality assurance from manufacturer/machine and service count
machine_key = """
SELECT  fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT CASE WHEN fk_service_type = 1 THEN id_service ELSE null END) AS num_fabric_production,
        COUNT(DISTINCT CASE WHEN fk_service_type = 2 THEN id_service ELSE null END) AS num_garment_production,
        COUNT(DISTINCT CASE WHEN fk_service_type = 3 THEN id_service ELSE null END) AS num_qualityassurance_production
FROM machine AS m
LEFT JOIN 
(
        SELECT *
        FROM service AS s
        LEFT JOIN service_type AS st ON s.fk_service_type = st.id_service_type
        WHERE s.verified = true AND s.active = true
) AS t ON m.fk_service = t.id_service
GROUP BY fk_manufacturer
ORDER BY fk_manufacturer;
"""
machine_key_df = helper.query_retrieval(machine_key, manufacturer_access)
machine_key_df[:5]
new_m_df = pd.merge(new_m_df, machine_key_df, on = 'id_manufacturer', how='left')
new_m_df[:5]
new_m_df.shape

(1300, 44)

#### External join showroom_visable, website link status and certificate compliance count number

In [14]:
# Manufacturer Showroom is visable or not
show_key = """
WITH show AS(SELECT  
        E.fk_entity AS id_manufacturer,
        visable AS showroom_visable,
        ROW_NUMBER() over (partition by E.fk_entity order by E.fk_entity)
FROM (
        SELECT distinct fk_entity,
                fk_feed_type,
                fk_entity_type,
                fk_feed_status,
                CASE 
                     WHEN visible_to_all = 'true' OR visible_to_watchlist = 'true' OR visible_to_customers = 'true' THEN 'True' 
                     ELSE 'False' 
                END AS visable
        FROM feed AS f
        LEFT JOIN feed_type AS ft ON f.fk_feed_type = ft.id_feed_type
        WHERE f.fk_feed_type = 1 and fk_feed_status=1 and fk_entity_type = 2
        ORDER BY fk_entity
        
) AS E
GROUP BY E.fk_entity,visable
ORDER BY E.fk_entity )   

SELECT id_manufacturer,
        showroom_visable
FROM show
WHERE row_number != 2
"""

show_key_df = helper.query_retrieval(show_key, shared_access)
new_m_df = pd.merge(new_m_df, show_key_df, on = 'id_manufacturer', how='left')
new_m_df.shape

(1300, 45)

In [15]:
# count certificate compliance number
compliance_key = """

SELECT 
        fk_manufacturer AS id_manufacturer,
        COUNT(DISTINCT mc.fk_manufacturer_compliance_type) AS num_compliance
FROM manufacturer_compliance AS mc
LEFT JOIN manufacturer_compliance_type AS mct ON mc.fk_manufacturer_compliance_type = mct.id_manufacturer_compliance_type
WHERE mc.active = True
GROUP BY fk_manufacturer
ORDER BY fk_manufacturer;

"""
compliance_key_df = helper.query_retrieval(compliance_key, manufacturer_access)
compliance_key_df[:10]
features = pd.merge(new_m_df, compliance_key_df, on = 'id_manufacturer', how='left')
features.shape

(1300, 46)

In [16]:
# Change the data type to int and replace Nan value to 0

list_int_change = ['po_min_qty', 'po_avg_qty', 'po_max_qty', 'on_stock_min',
        'on_stock_max', 'made_to_order_min', 'made_to_order_max',
        'sample_prototype_min', 'sample_prototype_max', 'brand_count',
        'count_parent_brands', 'count_parent_product_ranges',
        'sub_product_category_count', 'material_count_x',
        'material_type_count_x', 'material_count_y', 'material_type_count_y',
        'treatment_count', 'treatment_type_count', 'ref_brand_count',
        'num_fabric_production', 'num_garment_production',
        'num_qualityassurance_production', 'num_compliance','all_fields_percentage',
        'mandatory_fields_percentage','num_employee', 'revenue_per_year', 'pieces_per_year']

features[list_int_change] = features[list_int_change].fillna(0).astype('int')
features[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_city,manufacturer_country,manufacturer_status,manufacturer_type_filled,m_created_year_week,m_created_year_month,m_created_year,...,num_employee,revenue_per_year,pieces_per_year,ref_brand_count,has_description,num_fabric_production,num_garment_production,num_qualityassurance_production,showroom_visable,num_compliance
0,1,Foursource,yes,Berlin,Germany,Deactivated,yes,2017W05,2017M02,2017,...,60000,3000000089,4000000000,1,yes,0,0,0,NaN,1
1,2,Esstee Exports,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,900,0,3600000,9,yes,0,4,0,True,3
2,3,Esstee Exports,no,,India,Deactivated,no,2017W05,2017M02,2017,...,0,0,0,0,no,0,0,0,NaN,0
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Tirupur,India,Active,yes,2017W05,2017M02,2017,...,20000,300000000,95000000,13,yes,0,4,0,True,3
4,5,Saga Holdings,yes,Nanjing,China,Active,yes,2017W07,2017M02,2017,...,150,80000000,18000000,12,yes,0,0,0,NaN,2


In [17]:
#show all the column's data type 
features.columns.to_series().groupby(features.dtypes).groups

{dtype('int64'): Index(['id_manufacturer', 'contact_count', 'all_fields_percentage',
        'mandatory_fields_percentage', 'po_min_qty', 'po_avg_qty', 'po_max_qty',
        'count_parent_services', 'count_serives', 'on_stock_min',
        'on_stock_max', 'made_to_order_min', 'made_to_order_max',
        'sample_prototype_min', 'sample_prototype_max',
        'factory_location_count', 'brand_count', 'count_parent_brands',
        'count_parent_product_ranges', 'sub_product_category_count',
        'material_count_x', 'material_type_count_x', 'material_count_y',
        'material_type_count_y', 'treatment_count', 'treatment_type_count',
        'num_employee', 'revenue_per_year', 'pieces_per_year',
        'ref_brand_count', 'num_fabric_production', 'num_garment_production',
        'num_qualityassurance_production', 'num_compliance'],
       dtype='object'),
 dtype('O'): Index(['manufacturer_name', 'has_profile_image', 'manufacturer_city',
        'manufacturer_country', 'manufacturer_

In [18]:
# remove some categorial columns before doing one hot encoding 
features = features.drop(['manufacturer_city', 'manufacturer_country','m_created_year_week','m_created_year_month','m_created_year'],axis = 1)
features[:5]

,id_manufacturer,manufacturer_name,has_profile_image,manufacturer_status,manufacturer_type_filled,contact_count,all_fields_percentage,mandatory_fields_percentage,filled_mandatory_reached,po_min_qty,...,num_employee,revenue_per_year,pieces_per_year,ref_brand_count,has_description,num_fabric_production,num_garment_production,num_qualityassurance_production,showroom_visable,num_compliance
0,1,Foursource,yes,Deactivated,yes,4,77,67,False,600,...,60000,3000000089,4000000000,1,yes,0,0,0,NaN,1
1,2,Esstee Exports,yes,Active,yes,5,96,100,True,1000,...,900,0,3600000,9,yes,0,4,0,True,3
2,3,Esstee Exports,no,Deactivated,no,1,0,0,None,0,...,0,0,0,0,no,0,0,0,NaN,0
3,4,Eastman Exports Global Clothing Pvt Ltd,yes,Active,yes,7,96,100,True,3000,...,20000,300000000,95000000,13,yes,0,4,0,True,3
4,5,Saga Holdings,yes,Active,yes,3,92,100,True,500,...,150,80000000,18000000,12,yes,0,0,0,NaN,2


In [19]:
#drop id_manufacturer and manufacturer name
features = features.drop(['id_manufacturer','manufacturer_name'],axis = 1)

# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)
# Display the first 5 rows of the last 12 columns
features.head(5)

,contact_count,all_fields_percentage,mandatory_fields_percentage,po_min_qty,po_avg_qty,po_max_qty,count_parent_services,count_serives,on_stock_min,on_stock_max,...,manufacturer_status_Active,manufacturer_status_Deactivated,manufacturer_type_filled_no,manufacturer_type_filled_yes,filled_mandatory_reached_False,filled_mandatory_reached_True,has_description_no,has_description_yes,showroom_visable_False,showroom_visable_True
0,4,77,67,600,30000,1000000,2,7,0,0,...,0,1,0,1,1,0,0,1,0,0
1,5,96,100,1000,3000,50000,4,28,45,60,...,1,0,0,1,0,1,0,1,0,1
2,1,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,0,0
3,7,96,100,3000,50000,400000,2,7,50,70,...,1,0,0,1,0,1,0,1,0,1
4,3,92,100,500,2000,500000,4,27,47,59,...,1,0,0,1,0,1,0,1,0,0


In [20]:
#import the label file together (from Y.ipynb)
labels = pd.read_excel('labels.xlsx')

### Concatenate the feature and lables together

In [21]:
# Labels are the values we want to predict. Choose 1 label first
# 6 different labels
labels_np = np.array(labels).reshape(len(labels),6)

# Saving feature names for later use
feature_list = list(features.columns)
label_list = list(labels.columns)
# Convert to numpy array
features_np = np.array(features)

print("feature_shape", features_np.shape, "labels_np", labels_np.shape)

#concatenate x and y together 
total = np.concatenate((features_np, labels_np), axis = 1)
# total.shape
label_list

feature_shape (1300, 45) labels_np (1300, 6)


['nr_brand_watchlist',
 'nr_brand_supplierlist',
 'nr_viewer_buyer',
 'nr_buyers_sent_message',
 'nr_messages_received_from_buyers',
 'pv_count']

### Standardize to zero mean and unit variance. 

In [40]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
scaler = StandardScaler()

# Scale the data to be between -1 and 1
normalized_data = preprocessing.StandardScaler().fit_transform(total)


# # outliers_rows, outliers_columns = np.where(np.abs(normalized_data)>3)
# # remove the outlier column whose std > 3
# normalized_data = normalized_data[(np.abs(normalized_data) < 3).all(axis=1)]
# normalized_data.shape

features_np = total[:,0:-6]
labels_np = total[:,-6:]
labels_np.shape

/Users/glen/anaconda/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


(1300, 6)

### Find correlation between columns, remove the columns with high correlation 

In [23]:
# Create correlation matrix

corr_matrix = pd.DataFrame(features_np).corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
to_drop

[2, 9, 17, 21, 22, 34, 36, 38, 40, 42]

In [24]:
# remove the high correlated features column
drop_features = features.drop(features.columns[to_drop],axis = 1)
drop_features = np.array(drop_features)
drop_features

array([[   4,   77,  600, ...,    0,    0,    0],
       [   5,   96, 1000, ...,    0,    0,    1],
       [   1,    0,    0, ...,    1,    0,    0],
       ..., 
       [   1,   69, 1000, ...,    1,    0,    0],
       [   1,    4,    0, ...,    1,    0,    0],
       [   1,    4,    0, ...,    1,    0,    0]])

In [41]:
# remove the column name in features_list
to_drop_list = [feature_list[i] for i in to_drop]
drop_feature_list = [column for column in feature_list if column not in to_drop_list]
drop_feature_list[:5]

['contact_count',
 'all_fields_percentage',
 'po_min_qty',
 'po_avg_qty',
 'po_max_qty']

### Use Recursive Feature Elimination to reduce the feature

In [26]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE

svm = LinearSVC()
# create the RFE model for the svm classifier 
# and select attributes

rank_list = []
support_list = []
for i in range(6):
    # select 10 most influential features from RFE model
    rfe = RFE(svm, 10)
    rfe = rfe.fit(drop_features, labels_np[:,i])
    rfe.ranking_
    rank_list.append(rfe.ranking_)
    support_list.append(rfe.support_)
    i+=1
# print summaries for the selection of attributes
# support = rfe.support_

In [27]:
rank_list[0].shape

(35,)

In [28]:
drop_features.shape

(1300, 35)

In [29]:
# concatenate feature_list and rank together then sort by rank
# save as feature_rank.xlsx file
writer = pd.ExcelWriter('Final report.xlsx')
features.to_excel(writer,'features')
labels.to_excel(writer,'label')

label_list = ['nr_brand_watchlist',
 'nr_brand_supplierlist',
 'nr_viewer_buyer',
 'nr_buyers_sent_message',
 'nr_messages_receive_buyers',
 'pv_count']

f = np.array(drop_feature_list).reshape(len(drop_feature_list),1)

# 6 labels 
for i in range(6):
    r = rank_list[i].reshape(len(rank_list[i]),1)
    rank = np.concatenate((f, r), axis = 1)
    rank_pd = pd.DataFrame(rank,columns = ['feature','feature rank'])
    rank_pd[['feature rank']] = rank_pd[['feature rank']].apply(pd.to_numeric)
    rank_pd = rank_pd.sort_values(by=['feature rank'])
    rank_pd.to_excel(writer,label_list[i])

writer.save()

In [30]:
# the index of support_list 
np.where(support_list[0])[0]

array([ 0, 12, 14, 27, 28, 29, 30, 31, 32, 34])

In [31]:
# Store the selected feature name in the select_feature list
select_feature = []
select_feature_index = []
for i in range(6):
    select_ = np.array(feature_list)[np.where(support_list[i])]
    select_feature_index.append(np.where(support_list[i])[0].tolist())
    select_feature.append(select_)
select_feature[0]

array(['contact_count', 'sample_prototype_min', 'factory_location_count',
       'pieces_per_year', 'ref_brand_count', 'num_fabric_production',
       'num_garment_production', 'num_qualityassurance_production',
       'num_compliance', 'has_profile_image_yes'], 
      dtype='<U31')

In [32]:
select_feature_index[0]

[0, 12, 14, 27, 28, 29, 30, 31, 32, 34]

In [33]:
# selected features from 6 labels
select_feature

[array(['contact_count', 'sample_prototype_min', 'factory_location_count',
        'pieces_per_year', 'ref_brand_count', 'num_fabric_production',
        'num_garment_production', 'num_qualityassurance_production',
        'num_compliance', 'has_profile_image_yes'], 
       dtype='<U31'), array(['contact_count', 'factory_location_count',
        'count_parent_product_ranges', 'material_count_x',
        'pieces_per_year', 'ref_brand_count', 'num_fabric_production',
        'num_qualityassurance_production', 'num_compliance',
        'has_profile_image_yes'], 
       dtype='<U31'), array(['contact_count', 'sample_prototype_min', 'factory_location_count',
        'material_count_x', 'pieces_per_year', 'ref_brand_count',
        'num_fabric_production', 'num_garment_production',
        'num_qualityassurance_production', 'has_profile_image_yes'], 
       dtype='<U31'), array(['contact_count', 'sample_prototype_min',
        'count_parent_product_ranges', 'material_count_x',
        'piece

### Eliminate the feature data according to select_feature

In [50]:
# features_np: original features
# drop_features: features which remove high correlated column (>0.95)

(1300, 45)

In [84]:
# RFE_feature: features which don't remove high correlated column (>0.95) then choose the top 10 features from RFE
RFE_feature = features_np[:,select_feature_index[5]]
RFE_feature.shape

(1300, 10)

In [85]:
#LR_feature: features which remove high correlated column (>0.95) then choose the top 10 features from RFE
#0 : the first label
LR_feature = drop_features[:,select_feature_index[5]]
LR_feature.shape

(1300, 10)

### Split the data into training and testing

In [63]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features_np, labels_np[:,0], test_size = 0.25, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (975, 45)
Training Labels Shape: (975,)
Testing Features Shape: (325, 45)
Testing Labels Shape: (325,)


In [ ]:
"""
RFE_feature
LR_feature
features_np
drop_features
"""


### Implement on linear regression

In [89]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
lm = LinearRegression()
X = sm.add_constant(drop_features) # adding a constant
Y = labels_np[:,5]

# Note the difference in argument order
model = sm.OLS(Y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.8005
Date:                Mon, 06 Aug 2018   Prob (F-statistic):              0.791
Time:                        14:42:34   Log-Likelihood:                -6239.3
No. Observations:                1300   AIC:                         1.255e+04
Df Residuals:                    1264   BIC:                         1.274e+04
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6703      8.362      0.558      0.577     -11.735      21.076
x1             0.6178      1.338      0.462      0.644      -2.007       3.242
x2             0.0509      0.100      0.509      0.611      -0.145       0.247
x3             0.0010      0.001      1.880      0.060   -4.43e-05       0.002
x4         -2.086e-05   2.52e-05     -0.827      0.409   -7.04e-05    2.87e-05
x5         -1.488e-08      3e-08     -0.496      0.620   -7.37e-08    4.39e-08
x6            -1.1600      1.651     -0.703      0.482      -4.399       2.079
x7             0.4362      0.275      1.585      0.113      -0.104       0.976
x8             0.0032      0.004      0.835      0.404      -0.004       0.011
x9          7.666e-05      0.000      0.159      0.874      -0.001       0.001
x10           -0.0002      0.000     -0.721      0.471      -0.001       0.000
x11           -0.1662      0.131     -1.272      0.204      -0.423       0.090
x12            0.1115      0.085      1.314      0.189      -0.055       0.278
x13           -0.2834      1.244     -0.228      0.820      -2.724       2.157
x14           -0.0025      0.045     -0.056      0.956      -0.092       0.087
x15            0.0326      0.499      0.065      0.948      -0.946       1.011
x16            0.0067      0.028      0.241      0.810      -0.048       0.061
x17           -0.0457      0.095     -0.482      0.630      -0.232       0.140
x18            0.5461      1.596      0.342      0.732      -2.586       3.678
x19            0.0142      0.052      0.272      0.786      -0.088       0.117
x20            2.4376      2.773      0.879      0.380      -3.003       7.878
x21        -7.874e-05      0.001     -0.156      0.876      -0.001       0.001
x22        -1.003e-08   1.85e-08     -0.543      0.587   -4.63e-08    2.62e-08
x23         4.432e-09   1.54e-08      0.288      0.773   -2.57e-08    3.46e-08
x24           -0.3408      0.230     -1.479      0.139      -0.793       0.111
x25           -1.0872      2.324     -0.468      0.640      -5.647       3.473
x26           -1.0136      0.968     -1.047      0.296      -2.914       0.886
x27            7.7580      3.311      2.343      0.019       1.262      14.254
x28           -0.6389      1.645     -0.388      0.698      -3.867       2.589
x29           -1.3345      2.582     -0.517      0.605      -6.400       3.731
x30           -1.5830      3.752     -0.422      0.673      -8.943       5.777
x31            5.6341      5.235      1.076      0.282      -4.636      15.905
x32            3.3374      3.400      0.981      0.327      -3.334      10.009
x33           -0.6912      2.685     -0.257      0.797      -5.958       4.575
x34            1.5975      9.674      0.165      0.869     -17.381      20.576
x35            0.9304      2.086      0.446      0.656      -3.162       5.023
==

In [102]:
mu, sigma = 0, 1 # mean and standard deviation
s = np.random.normal(mu, sigma, 1300).reshape(1300,1)
s

array([[-0.96417193],
       [-0.8829177 ],
       [ 0.667489  ],
       ..., 
       [-0.89837797],
       [ 0.46617441],
       [ 0.60274902]])

In [103]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
lm = LinearRegression()
X = sm.add_constant(drop_features) # adding a constant
Y = s

# Note the difference in argument order
model = sm.OLS(Y, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.7602
Date:                Mon, 06 Aug 2018   Prob (F-statistic):              0.843
Time:                        15:57:17   Log-Likelihood:                -1850.8
No. Observations:                1300   AIC:                             3774.
Df Residuals:                    1264   BIC:                             3960.
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.4677      0.286     -1.636      0.102      -1.029       0.093
x1            -0.0252      0.046     -0.550      0.582      -0.115       0.065
x2             0.0024      0.003      0.709      0.478      -0.004       0.009
x3           1.39e-07   1.86e-05      0.007      0.994   -3.64e-05    3.67e-05
x4           4.84e-07   8.63e-07      0.561      0.575   -1.21e-06    2.18e-06
x5         -1.337e-09   1.02e-09     -1.304      0.192   -3.35e-09    6.74e-10
x6            -0.0354      0.056     -0.626      0.531      -0.146       0.075
x7             0.0021      0.009      0.221      0.825      -0.016       0.021
x8          4.029e-05      0.000      0.305      0.760      -0.000       0.000
x9          2.524e-06   1.65e-05      0.153      0.879   -2.99e-05    3.49e-05
x10        -4.605e-06   7.98e-06     -0.577      0.564   -2.03e-05    1.11e-05
x11           -0.0029      0.004     -0.656      0.512      -0.012       0.006
x12            0.0026      0.003      0.904      0.366      -0.003       0.008
x13            0.0291      0.043      0.684      0.494      -0.054       0.113
x14            0.0013      0.002      0.828      0.408      -0.002       0.004
x15            0.0025      0.017      0.147      0.883      -0.031       0.036
x16            0.0007      0.001      0.774      0.439      -0.001       0.003
x17            0.0001      0.003      0.039      0.969      -0.006       0.006
x18           -0.0391      0.055     -0.717      0.474      -0.146       0.068
x19           -0.0003      0.002     -0.180      0.857      -0.004       0.003
x20            0.0695      0.095      0.733      0.464      -0.117       0.256
x21        -1.368e-06   1.72e-05     -0.080      0.937   -3.51e-05    3.24e-05
x22        -7.575e-10   6.31e-10     -1.200      0.230      -2e-09    4.81e-10
x23         3.831e-10   5.26e-10      0.728      0.466   -6.49e-10    1.41e-09
x24           -0.0026      0.008     -0.327      0.744      -0.018       0.013
x25            0.0609      0.079      0.767      0.443      -0.095       0.217
x26           -0.0151      0.033     -0.455      0.649      -0.080       0.050
x27           -0.1408      0.113     -1.243      0.214      -0.363       0.081
x28           -0.0506      0.056     -0.899      0.369      -0.161       0.060
x29            0.0295      0.088      0.334      0.739      -0.144       0.203
x30            0.1869      0.128      1.457      0.145      -0.065       0.439
x31           -0.0385      0.179     -0.215      0.830      -0.390       0.313
x32            0.2453      0.116      2.110      0.035       0.017       0.473
x33            0.1027      0.092      1.119      0.263      -0.077       0.283
x34           -0.2446      0.331     -0.740      0.460      -0.894       0.404
x35            0.0201      0.071      0.282      0.778      -0.120       0.160
==

<style>
.alert-green {
  color: rgb(60,118,61) !important;
}
</style>

<style>
.alert-green {
  color: rgb(60,118,61) !important;
}
</style>